<a href="https://colab.research.google.com/github/TokaAyman/NLP_ITI/blob/main/lab2_NER.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install datasets


In [2]:
from datasets import load_dataset

In [3]:
dataset = load_dataset("eriktks/conll2003")


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [4]:
dataset

DatasetDict({
    train: Dataset({
        features: ['id', 'tokens', 'pos_tags', 'chunk_tags', 'ner_tags'],
        num_rows: 14041
    })
    validation: Dataset({
        features: ['id', 'tokens', 'pos_tags', 'chunk_tags', 'ner_tags'],
        num_rows: 3250
    })
    test: Dataset({
        features: ['id', 'tokens', 'pos_tags', 'chunk_tags', 'ner_tags'],
        num_rows: 3453
    })
})

In [13]:
!python -m spacy init config config.cfg --lang en --pipeline ner

⚠ To generate a more effective transformer-based config (GPU-only),
install the spacy-transformers package and re-run this command. The config
generated now does not use transformers.
ℹ Generated config template specific for your use case
- Language: en
- Pipeline: ner
- Optimize for: efficiency
- Hardware: CPU
- Transformer: None
✔ Auto-filled config with all values
✔ Saved config
config.cfg
You can now add your data and train your pipeline:
python -m spacy train config.cfg --paths.train ./train.spacy --paths.dev ./dev.spacy


example of the needed shape of data

In [ ]:
TRAIN_DATA = [
("Apple Inc. is located in Cupertino", {"entities": [(0, 10, "ORG"), (25, 35, "LOC")]}),
("Tim Cook is the CEO of Apple", {"entities": [(0, 8, "PER"), (23, 28, "ORG")]})]

In [6]:
label_names = dataset['train'].features['ner_tags'].feature.names

In [7]:
label_names

['O', 'B-PER', 'I-PER', 'B-ORG', 'I-ORG', 'B-LOC', 'I-LOC', 'B-MISC', 'I-MISC']

In [8]:
def convert_to_spacy(data, output_path):
    db = DocBin()
    for sample in data:
        words = sample['tokens']
        labels = sample['ner_tags']
        doc = nlp.make_doc(" ".join(words))
        ents = []
        offset = 0
        for word, label in zip(words, labels):
            start = offset
            end = start + len(word)
            label_name = label_names[label]
            if label_name != "O":
                ents.append(doc.char_span(start, end, label=label_name))
            offset += len(word) + 1  # +1 for space
        doc.ents = [e for e in ents if e is not None]
        db.add(doc)
    db.to_disk(output_path)

In [9]:
import spacy
from spacy.tokens import DocBin

nlp = spacy.blank("en")


In [10]:
convert_to_spacy(dataset["train"], "train.spacy")
convert_to_spacy(dataset["validation"], "dev.spacy")

In [15]:
!python -m spacy train config.cfg --output ./output


ℹ Saving to output directory: output
ℹ Using CPU
ℹ To switch to GPU 0, use the option: --gpu-id 0

=========================== Initializing pipeline ===========================
✔ Initialized pipeline

============================= Training pipeline =============================
ℹ Pipeline: ['tok2vec', 'ner']
ℹ Initial learn rate: 0.001
E    #       LOSS TOK2VEC  LOSS NER  ENTS_F  ENTS_P  ENTS_R  SCORE 
---  ------  ------------  --------  ------  ------  ------  ------
  0       0          0.00     48.26    0.00    0.00    0.00    0.00
  0     200         87.63   3469.77   57.82   59.76   56.00    0.58
  0     400        322.63   2210.70   74.16   75.01   73.32    0.74
  0     600        306.83   2083.39   78.65   79.82   77.52    0.79
  0     800        312.63   2115.00   82.17   83.17   81.19    0.82
  0    1000        357.05   2230.82   83.30   84.18   82.44    0.83
  1    1200        517.91   2187.74   86.02   87.23   84.84    0.86
  1    1400        410.69   1650.70   86.32   86.4

Evaluation

In [20]:
import spacy
nlp = spacy.load("output/model-best")
doc = nlp("Tim Cook is the CEO of Apple")
print([(ent.text, ent.label_) for ent in doc.ents])


[('Tim', 'B-PER'), ('Cook', 'I-PER'), ('Apple', 'B-ORG')]
